## VISUALIZATION PROJECT Geospatial Business Intelligence (BI)


In [1]:
#Tras iniciar servidor de DB mongo, importo módulo 'pymongo'.
import pymongo
import pandas as pd
import numpy as np
#Asigno variable al cliente de DB y a la DB.
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [2]:
#Verifico que se han importado todos los documentos.
cursor = db.companies.find()
data = list(cursor)
print(len(data))

18801


In [3]:
def q_qty_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return len(list(map(lambda r:r,list(q))))
atr1 = 'category_code'
var1 = 'web'
print(q_qty_1(atr1,var1))

3787


In [6]:
def q_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return list(map(lambda r:[(r['name']),(r['offices'])],list(q)))
atr1 = 'category_code'
var1 = 'web'
#display(q_1(atr1,var1))

In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

In [ ]:
def convertRestaurant(restaurant):
    return {
        "name": restaurant["name"],
        "cuisine":restaurant["cuisine"],
        "position": toGeoJSON(restaurant["address"]["coord"])
    }


In [18]:
#cursor = 
df = pd.DataFrame(data)
#display(df.dtypes)
columns = df.dtypes
print(columns)

_id                     object
acquisition             object
acquisitions            object
alias_list              object
blog_feed_url           object
blog_url                object
category_code           object
competitions            object
created_at              object
crunchbase_url          object
deadpooled_day         float64
deadpooled_month       float64
deadpooled_url          object
deadpooled_year        float64
description             object
email_address           object
external_links          object
founded_day            float64
founded_month          float64
founded_year           float64
funding_rounds          object
homepage_url            object
image                   object
investments             object
ipo                     object
milestones              object
name                    object
number_of_employees    float64
offices                 object
overview                object
partners                object
permalink               object
phone_nu

In [19]:
g = df.columns.to_series().groupby(df.dtypes).groups
{k.name: v for k, v in g.items()}

{'float64': Index(['deadpooled_day', 'deadpooled_month', 'deadpooled_year', 'founded_day',
        'founded_month', 'founded_year', 'number_of_employees'],
       dtype='object'),
 'object': Index(['_id', 'acquisition', 'acquisitions', 'alias_list', 'blog_feed_url',
        'blog_url', 'category_code', 'competitions', 'created_at',
        'crunchbase_url', 'deadpooled_url', 'description', 'email_address',
        'external_links', 'funding_rounds', 'homepage_url', 'image',
        'investments', 'ipo', 'milestones', 'name', 'offices', 'overview',
        'partners', 'permalink', 'phone_number', 'products', 'providerships',
        'relationships', 'screenshots', 'tag_list', 'total_money_raised',
        'twitter_username', 'updated_at', 'video_embeds'],
       dtype='object')}

In [25]:
#Columnas originales
col_float = ['deadpooled_day','deadpooled_month','deadpooled_year','founded_day','founded_month',
             'founded_year','number_of_employees']
col_obj = ['_id','acquisition','acquisitions','alias_list','blog_feed_url','blog_url','category_code',
           'competitions','created_at','crunchbase_url','deadpooled_url','description','email_address',
        'external_links','funding_rounds','homepage_url','image','investments','ipo','milestones', 
           'name','offices','overview','partners','permalink','phone_number','products','providerships',
        'relationships','screenshots','tag_list','total_money_raised','twitter_username','updated_at',
           'video_embeds']
col_drop1 = ['deadpooled_day','deadpooled_month','founded_day','founded_month','alias_list','blog_feed_url',
             'blog_url','created_at','crunchbase_url','deadpooled_url','email_address','external_links',
            'image','permalink','phone_number','screenshots','tag_list','updated_at','video_embeds']
col_drop2 = ['acquisition','acquisitions','competitions','description','funding_rounds','homepage_url',
             'investments','ipo','milestones','partners','products','providerships','relationships',
             'twitter_username']
col_ok = ['deadpooled_year','founded_year','number_of_employees','_id','category_code','name','offices',
          'overview','total_money_raised']

print(len(col_float)+len(col_obj),len(col_drop1)+len(col_drop2)+len(col_ok))

42 42


In [53]:
df_drop1 = df.drop(col_drop1, axis=1)
df_drop2 = df_drop1.drop(col_drop2, axis=1)
df_deadnull = df_drop2[df_drop2['deadpooled_year'].isnull()]
df_nulls = df_deadnull.dropna(subset=['number_of_employees'])
display(len(df_nulls))

8403

In [60]:
#Elimino rows que no me aportan nada
df_values = df_nulls[df_nulls['offices'].map(len) > 0]
df_values = df_values[df_values.number_of_employees != 0]
print(len(df_values))
display(df_values.head(10))

6943


,_id,category_code,deadpooled_year,founded_year,name,number_of_employees,offices,overview,total_money_raised
5,52cdef7c4bab8bd675297d8d,news,NaN,2004.0,Digg,60.0,"[{'description': None, 'address1': '135 Missis...",<p>Digg is a user driven social content websit...,$45M
6,52cdef7c4bab8bd675297d91,web,NaN,2006.0,Geni,18.0,"[{'description': 'Headquarters', 'address1': '...",<p>Geni is an online community of casual and e...,$16.5M
11,52cdef7c4bab8bd675297d97,news,NaN,2007.0,Scribd,50.0,"[{'description': 'HQ', 'address1': '539 Bryant...",<p>Scribd is the world’s biggest digital libra...,$25.8M
12,52cdef7c4bab8bd675297d94,social,NaN,2006.0,Twitter,1300.0,"[{'description': '', 'address1': '1355 Market ...","<p>Created in 2006, Twitter is a global real-t...",$1.16B
15,52cdef7c4bab8bd675297d9c,social,NaN,2007.0,MeetMoi,15.0,"[{'description': None, 'address1': '', 'addres...","<p>MeetMoi, the leading location-based mobile ...",$5.58M
20,52cdef7c4bab8bd675297da1,web,NaN,2002.0,Plaxo,50.0,"[{'description': 'HQ', 'address1': '1050 Enter...",<p>Plaxo helps keep people connected by solvin...,$28.3M
22,52cdef7c4bab8bd675297d8e,social,NaN,2004.0,Facebook,5299.0,"[{'description': 'Headquarters', 'address1': '...",<p>Facebook is the world&#8217;s largest socia...,$2.43B
23,52cdef7c4bab8bd675297d9b,web,NaN,1995.0,eBay,15000.0,"[{'description': 'Headquarters', 'address1': '...","<p>Founded in 1995 in San Jose, CA, eBay conne...",$6.7M
24,52cdef7c4bab8bd675297da4,search,NaN,2006.0,Powerset,60.0,"[{'description': None, 'address1': '475 Branna...",<p>Powerset is a search engine focused on natu...,$22.5M
26,52cdef7c4bab8bd675297da5,advertising,NaN,2002.0,Technorati,35.0,"[{'description': '', 'address1': '360 Post St....",<p>Technorati Media is an advertising platform...,$32.1M


In [64]:
#Data frames por tipo de empresas
col_type = ['games_video','web','software']
df_type = df_values[df_values['category_code'].isin(col_type)]\
.sort_values(by=['category_code','number_of_employees'])
print(len(df_type))
display(df_type.head(10))

3213


,_id,category_code,deadpooled_year,founded_year,name,number_of_employees,offices,overview,total_money_raised
485,52cdef7c4bab8bd675297f6b,games_video,NaN,2007.0,Contrastream,1.0,"[{'description': None, 'address1': '62 Sexton ...",<p>ContraStream provides socially filtered ind...,$0
1663,52cdef7c4bab8bd67529840c,games_video,NaN,2007.0,Sporting Connections,1.0,"[{'description': None, 'address1': '7/43 Dalto...",<p>SportingConnections is a free sports based ...,$0
2448,52cdef7c4bab8bd675298718,games_video,NaN,2007.0,Myplaylist,1.0,"[{'description': None, 'address1': '', 'addres...",<p>Myplaylist is a picture / music playlist co...,$0
2953,52cdef7d4bab8bd675298916,games_video,NaN,2008.0,twit+,1.0,"[{'description': None, 'address1': '', 'addres...",<p>twit+ works on top of Twitter to let users ...,$0
3067,52cdef7d4bab8bd675298988,games_video,NaN,NaN,GPokr,1.0,"[{'description': None, 'address1': '', 'addres...","<p><a href=""http://gpokr.com"" title=""GPokr"" re...",$0
3597,52cdef7d4bab8bd675298b97,games_video,NaN,2000.0,sph3re,1.0,"[{'description': None, 'address1': '11 rue Phi...",<p>sph3re.tv is a video portal dedicated to an...,$0
3981,52cdef7d4bab8bd675298d18,games_video,NaN,2006.0,Jungle Jam Tv,1.0,"[{'description': None, 'address1': '', 'addres...",<p>Video guides of buildings and landmarks on ...,$0
4096,52cdef7d4bab8bd675298d8d,games_video,NaN,2008.0,Faneebo,1.0,"[{'description': 'Faneebo Headquarters', 'addr...",<p>Faneebo.com is an exclusive music fan club ...,$0
5212,52cdef7d4bab8bd6752991e6,games_video,NaN,2008.0,World Wide Talent Search,1.0,"[{'description': '', 'address1': 'N/A', 'addre...",<p>World Wide Talent Search connects music lov...,$0
5439,52cdef7d4bab8bd6752992cc,games_video,NaN,2007.0,Sportstation,1.0,"[{'description': '', 'address1': '', 'address2...",<p>Sportstation specialises in providing websi...,$0


In [ ]:
#Esto es para buscar con regex de mongo
#{ name: { $in: [ /^key./i ] } }    { tag_list: { $in: [ /.games./i ] } }  
#{ updated_at: { $in: [ /2008$/ ] } } 
#Para descartar arrays vacios en mongo
#{offices:{ $gt: [] } } (hay otras opciones en un link en pendientes)